# This notebook is trying to explain the history of a country by itself
## Afghanistan

In [12]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

asylum = pd.read_csv('.\\Data\\Clean\\Asylum_data.csv')
asylum

,country_of_origin_abbr,country_of_origin_name,country_of_asylum_abbr,country_of_asylum_name,region_of_asylum,category,year,count
0,DZA,Algeria,MLT,Malta,Europe,Refugee,1962,5
1,AGO,Angola,COD,Dem. Rep. of the Congo,Southern Africa,Refugee,1962,20000
2,AGO,Angola,NAM,Namibia,Southern Africa,Refugee,1962,277
3,BDI,Burundi,NAM,Namibia,Southern Africa,Refugee,1962,13
4,CHN,China,NPL,Nepal,Asia and the Pacific,Refugee,1962,5
...,...,...,...,...,...,...,...,...
103303,ZWE,Zimbabwe,POL,Poland,Europe,Asylum-seekers,2024,9
103304,ZWE,Zimbabwe,ZAF,South Africa,Southern Africa,Asylum-seekers,2024,89
103305,ZWE,Zimbabwe,SWE,Sweden,Europe,Asylum-seekers,2024,5
103306,ZWE,Zimbabwe,USA,United States of America,Americas,Asylum-seekers,2024,641


In [13]:
# Timeline of the afghanistan migration
def Get_Destination_by_year_df(df: pd.DataFrame, country_abbr: str) -> pd.DataFrame:
    result = df[df['country_of_origin_abbr'] == country_abbr]
    result = result.groupby(['year', 'country_of_asylum_abbr']).agg({'count' : 'sum'}).reset_index()
    result['merge_column'] = result['country_of_asylum_abbr'] + result['year'].astype(str)
    return result

def Get_countries_and_years_df(df: pd.DataFrame) -> pd.DataFrame:
    countries = df['country_of_asylum_abbr'].unique()
    years = df['year'].unique()
    result = pd.DataFrame(columns=["country", "year"])
    for country in countries:
        c_list = [country] * len(years)
        country_list_df = pd.DataFrame({"country" : c_list, "year" : years})
        result = pd.concat([result, country_list_df], ignore_index=True)
    result['merge_column'] = result['country'] + result['year'].astype(str)
    return result

## TODO: For loop at group by with if statement


def Merge_and_clean_df(destination_df: pd.DataFrame, years_df: pd.DataFrame) -> pd.DataFrame:
    final = years_df.merge(destination_df, on='merge_column', how='left')
    final[final['year_x'] != final['year_y']]
    final = final.drop(columns=['year_y', 'merge_column', 'country_of_asylum_abbr'])
    final = final.rename(columns={'year_x': 'year'})
    final['count'] = final['count'].fillna(0)
    final['cumulative_sum'] = final.groupby('country')['count'].cumsum()
    return final

def Get_ready_for_plot_df(df: pd.DataFrame, country_of_origin_abbr: str) -> pd.DataFrame:
    destination = Get_Destination_by_year_df(df, country_of_origin_abbr)
    yearly = Get_countries_and_years_df(destination)
    return Merge_and_clean_df(destination, yearly)    

def Get_total_country_migration_df(df: pd.DataFrame, country_of_origin_abbr: str) -> pd.DataFrame:
    return df[df['country_of_origin_abbr'] == country_of_origin_abbr].groupby('year').agg({'count' : 'sum'}).reset_index()

afghanistan = asylum[asylum['country_of_origin_abbr'] == 'VEN']
afghanistan.groupby('year').agg({'count' : 'sum'}).reset_index()

afghanistan

,country_of_origin_abbr,country_of_origin_name,country_of_asylum_abbr,country_of_asylum_name,region_of_asylum,category,year,count
556,VEN,Venezuela (Bolivarian Republic of),CHE,Switzerland,Europe,Asylum-seekers,1973,5
3311,VEN,Venezuela (Bolivarian Republic of),FRA,France,Europe,Asylum-seekers,1984,5
3312,VEN,Venezuela (Bolivarian Republic of),DEU,Germany,Europe,Asylum-seekers,1984,5
4391,VEN,Venezuela (Bolivarian Republic of),DEU,Germany,Europe,Asylum-seekers,1986,5
6490,VEN,Venezuela (Bolivarian Republic of),CAN,Canada,Americas,Asylum-seekers,1989,50
...,...,...,...,...,...,...,...,...
103235,VEN,Venezuela (Bolivarian Republic of),CHE,Switzerland,Europe,Asylum-seekers,2024,41
103236,VEN,Venezuela (Bolivarian Republic of),SXM,Sint Maarten (Dutch part),Americas,Asylum-seekers,2024,5
103237,VEN,Venezuela (Bolivarian Republic of),URY,Uruguay,Americas,Asylum-seekers,2024,444
103238,VEN,Venezuela (Bolivarian Republic of),URY,Uruguay,Americas,Other people in need of international protection,2024,775


# Show migration history

In [21]:
fig = px.line(Get_total_country_migration_df(asylum, 'ITA'), x='year', y='count')
fig.show()

In [15]:
asylum
d = pd.DataFrame({"col": []})
u = []

u.append(asylum['year'])
u.append(asylum['year'])
pd.concat(u, ignore_index=True).count()

np.int64(206616)

# Show countries they migrated to

In [16]:
Get_Destination_by_year_df(asylum, 'VEN')

,year,country_of_asylum_abbr,count,merge_column
0,1973,CHE,5,CHE1973
1,1984,DEU,5,DEU1984
2,1984,FRA,5,FRA1984
3,1986,DEU,5,DEU1986
4,1989,CAN,50,CAN1989
...,...,...,...,...
628,2024,SUR,26,SUR2024
629,2024,SWE,25,SWE2024
630,2024,SXM,5,SXM2024
631,2024,URY,1219,URY2024


In [23]:
destination = Get_Destination_by_year_df(asylum, 'ITA')

fig = px.bar(destination[destination['year'] >= 2016], x='year', y='count', color='country_of_asylum_abbr')
fig.show()

# Population that left the country 

In [18]:
# this is just for heatmap type plot
final_heat = final[final['cumulative_sum'] != 0]
print(final_heat[final_heat['country'] == 'VEN'])

NameError: name 'final' is not defined

# Testing the results of the functions

In [ ]:
des.equals(Get_Destination_by_year_df(asylum, 'VEN'))

True

In [ ]:
dd.equals(Add_every_year(des))

True

In [ ]:
final.equals(Get_ready_for_plot_df(asylum, 'VEN'))

True

In [ ]:
misk[misk['merge_column'] == False]

,year,country_of_asylum_abbr,count,merge_column


# Plotting a map that shows country of destination

In [22]:
final = Get_ready_for_plot_df(asylum, 'ITA')

fig = px.scatter_geo(final, locations="country", locationmode='ISO-3', color="country",
                     hover_name="country", size="cumulative_sum",
                     animation_frame="year",
                     projection="natural earth")

fig.show()

In [ ]:
final_heat = final[final['cumulative_sum'] != 0]

fig = px.choropleth(final_heat, locations="country", locationmode='ISO-3',
                     color="cumulative_sum",
                     color_continuous_scale="Viridis",
                     hover_name="country",
                     animation_frame="year",
                     projection="natural earth")

fig.show()

In [ ]:
final[2:]

,country,year,count,cumulative_sum
2,FRA,1991,0.0,5.0
3,FRA,1992,5.0,10.0
4,FRA,1993,22.0,32.0
5,FRA,1994,42.0,74.0
6,FRA,1995,36.0,110.0
...,...,...,...,...
2983,TZA,2020,0.0,0.0
2984,TZA,2021,0.0,0.0
2985,TZA,2022,0.0,0.0
2986,TZA,2023,0.0,0.0


In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=des['country_of_asylum_abbr'],
    z=des['count'].astype(float),
    locationmode='ISO-3',
    colorscale='Reds',
    autocolorscale=False,
    text=des['country_of_asylum_abbr'], # hover text
    marker_line_color='white', # line markers between states
    colorbar=dict(
        title=dict(
            text="Millions USD"
            )
    )
))

fig.update_layout(
    title_text='Destination countries for Afghanistan migrants',
    geo = dict(
        
        projection=go.layout.geo.Projection(type = 'equal earth'),
        showlakes=False),
)

fig.show()

In [ ]:
final_heat

,country,year,count,cumulative_sum
0,CHE,1973,5.0,5.0
1,CHE,1984,0.0,5.0
2,CHE,1986,0.0,5.0
3,CHE,1989,0.0,5.0
4,CHE,1990,0.0,5.0
...,...,...,...,...
2612,SXM,2024,5.0,23.0
2648,KGZ,2021,5.0,5.0
2649,KGZ,2022,0.0,5.0
2650,KGZ,2023,0.0,5.0


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
df['text'] = df['name'] + '<br>Population ' + (df['pop']/1e6).astype(str)+' million'
df.head()

,name,pop,lat,lon,text
0,New York,8287238,40.730599,-73.986581,New York <br>Population 8.287238 million
1,Los Angeles,3826423,34.053717,-118.242727,Los Angeles <br>Population 3.826423 million
2,Chicago,2705627,41.875555,-87.624421,Chicago <br>Population 2.705627 million
3,Houston,2129784,29.758938,-95.367697,Houston <br>Population 2.129784 million
4,Philadelphia,1539313,39.952335,-75.163789,Philadelphia <br>Population 1.539313 million


In [ ]:


df['text'] = df['name'] + '<br>Population ' + (df['pop']/1e6).astype(str)+' million'
limits = [(0,50000),(50000,250000),(250000,500000),(500000,1000000),(1000000,4000000)] # limits of the colors
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []
scale = 5000

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()